<h1>Lab | Extending the internal databases with audio features</h1>

At this point, you have the **hot_songs** and the **not_hot_songs** databases. However, you don't have any acoustic information about the songs. The purpose of this lab is to use Spotify's API to extend both databases with this information for each song included to use this information later.

**Instructions**

1) Create a function to search a given **single** song in the Spotify API: **search_song(title, artist)**.
Take into account that sometimes Spotify's API will return several matches for the same song title (different artists, a different album of the same artist, version of the song,...etc). Then it will be nice to display a list of outputs to the user and let him/her select which is the right match. Once the desired song is located, the function should return the href/id/uri of the song to the code (not to the user). Keep in mind, that a given song might not be available on Spotify's API (make sure to use the song's title and artist searching the song). If the song is not found, the function must return an empty string as the href/id/uri. Also, in this case, you should remove this song from the database.

2) Create a function **get_audio_features(list_of_songs)** to obtain the audio features of a given list of songs (the content of list_of_songs can be the href/id/uri).
Be careful to not exceed the number of calls to the API otherwise, you will be banned and you will have to wait several hours before launching a new request see here. A good strategy to prevent this problem is to split the list of song id's in "chunks" of 50 songs id's and wait 20 seconds before asking for the audio features of the next "chunk. Then, use this function to create a Pandas Dataframe with the audio features of the list of songs. Hint: create a dictionary with the song's audio features as keys and an **empty list as values**. Then fill in the lists with the corresponding audio features of each song. Finally, create your data frame from the dictionary.

3) Once the previous function has been created, create another function **add_audio_features(df, audio_features_df)** to concat a given dataframe with the dataframe containing the audio features alongside any other desired info, and return the extended data frame. Replace the old internal files of songs (hot and not hot) with the extended data frames with the audio features and save them into separate files on the disk.

In [1]:
!pip install spotipy tqdm

<h2>Part 1</h2>

In [2]:
import sys
sys.path.insert(1, '/content/drive/MyDrive/Credentials/')
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import math
import tqdm # to see the progress

In [3]:
def search_song(title, artist, ask_for_options=True):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))
    print("Searching for song {} of artist {}".format(title,artist))
    results = sp.search(q="track:"+title+" artist:"+artist,limit=10)
    if ask_for_options == True:
        if len(results['tracks']['items']) > 1:
            count = 0
            for item in results['tracks']['items']:
                print(count,"Song: '{}', artist: '{}', album: '{}', duration: {} minutes".format(item['name'], item['artists'][0]['name'],item['album']['name'], round((item['duration_ms']/60000),2)))
                count = count + 1
            song_index = int(input("Desirable song (row number): "))
            return results['tracks']['items'][song_index]['uri'] 

        elif len(results['tracks']['items']) == 1:
            return results['tracks']['items'][0]['uri'] 
        else:
            return ''
    
    if ask_for_options == False:
        if len(results['tracks']['items']) >= 1:
            return results['tracks']['items'][0]['uri']
        else:
            return ''

Checking:

In [4]:
# test 1
search_song('scientist', 'coldplay',)

Searching for song scientist of artist coldplay
0 Song: 'The Scientist', artist: 'Coldplay', album: 'A Rush of Blood to the Head', duration: 5.16 minutes
1 Song: 'The Scientist', artist: 'Coldplay', album: 'The Scientist', duration: 5.19 minutes
2 Song: 'The Scientist - Live in Buenos Aires', artist: 'Coldplay', album: 'Live in Buenos Aires', duration: 6.48 minutes
3 Song: 'The Scientist', artist: 'Coldplay', album: '100 Greatest Acoustic Songs', duration: 5.16 minutes
4 Song: 'The Scientist', artist: 'Coldplay', album: 'Chilled Acoustic', duration: 5.16 minutes
5 Song: 'The Scientist', artist: 'Coldplay', album: 'A Thousand Years - Heartbreak Pop', duration: 5.16 minutes
6 Song: 'The Scientist', artist: 'Coldplay', album: '00's Mellow Gold', duration: 5.16 minutes
7 Song: 'The Scientist', artist: 'Coldplay', album: 'Mellow Hits', duration: 5.16 minutes
8 Song: 'The Scientist', artist: 'Coldplay', album: '100 Greatest Dinner Party Songs', duration: 5.16 minutes
9 Song: 'The Scientist',

'spotify:track:3eWIQ3LwfMcfYZzbGa6CUd'

In [5]:
# test 1.5
search_song('scientist', 'coldplay', ask_for_options=False)

Searching for song scientist of artist coldplay


'spotify:track:75JFxkI2RXiU7L9VXzMkle'

In [6]:
# test 2
print(search_song('gfhtdjht', 'coldplay'))

Searching for song gfhtdjht of artist coldplay



In [7]:
# test 3
search_song('Tales Of Anicca', 'Lea Valentina')

Searching for song Tales Of Anicca of artist Lea Valentina


'spotify:track:0RcGrOHL67qY8doQTchmCS'

<h2>Part 2</h2>

Getting audio features of a song(using song uri):

In [8]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))

In [9]:
sp.audio_features(['spotify:track:75JFxkI2RXiU7L9VXzMkle'])

[{'danceability': 0.557,
  'energy': 0.442,
  'key': 5,
  'loudness': -7.224,
  'mode': 1,
  'speechiness': 0.0243,
  'acousticness': 0.731,
  'instrumentalness': 1.46e-05,
  'liveness': 0.11,
  'valence': 0.213,
  'tempo': 146.277,
  'type': 'audio_features',
  'id': '75JFxkI2RXiU7L9VXzMkle',
  'uri': 'spotify:track:75JFxkI2RXiU7L9VXzMkle',
  'track_href': 'https://api.spotify.com/v1/tracks/75JFxkI2RXiU7L9VXzMkle',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/75JFxkI2RXiU7L9VXzMkle',
  'duration_ms': 309600,
  'time_signature': 4}]

Creating a list of random uris:

In [10]:
song_uris = ['spotify:track:0g7cOz3g27daGHxymyOEHt','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ','spotify:track:0RcGrOHL67qY8doQTchmCS', 'spotify:track:3EfldjXLMDUU6zYUAfEoGz', 'spotify:track:4ClJlECmc0Yfxz6IiI3sye', 'spotify:track:5Lv60aUIxDRhnNXrcECgit', 'spotify:track:198Kj3dapHMkGttkRPdlPZ']

Scraping audio features of the songs from this list:

In [11]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))
results = sp.audio_features(song_uris)
pd.DataFrame(results)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,0g7cOz3g27daGHxymyOEHt,spotify:track:0g7cOz3g27daGHxymyOEHt,https://api.spotify.com/v1/tracks/0g7cOz3g27da...,https://api.spotify.com/v1/audio-analysis/0g7c...,309600,4
1,0.440,0.250,5,-16.942,0,0.0604,0.974,0.968000,0.128,0.166,133.807,audio_features,0RcGrOHL67qY8doQTchmCS,spotify:track:0RcGrOHL67qY8doQTchmCS,https://api.spotify.com/v1/tracks/0RcGrOHL67qY...,https://api.spotify.com/v1/audio-analysis/0RcG...,482125,4
2,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,3EfldjXLMDUU6zYUAfEoGz,spotify:track:3EfldjXLMDUU6zYUAfEoGz,https://api.spotify.com/v1/tracks/3EfldjXLMDUU...,https://api.spotify.com/v1/audio-analysis/3Efl...,309600,4
3,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,4ClJlECmc0Yfxz6IiI3sye,spotify:track:4ClJlECmc0Yfxz6IiI3sye,https://api.spotify.com/v1/tracks/4ClJlECmc0Yf...,https://api.spotify.com/v1/audio-analysis/4ClJ...,309600,4
4,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,5Lv60aUIxDRhnNXrcECgit,spotify:track:5Lv60aUIxDRhnNXrcECgit,https://api.spotify.com/v1/tracks/5Lv60aUIxDRh...,https://api.spotify.com/v1/audio-analysis/5Lv6...,309600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.440,0.250,5,-16.942,0,0.0604,0.974,0.968000,0.128,0.166,133.807,audio_features,0RcGrOHL67qY8doQTchmCS,spotify:track:0RcGrOHL67qY8doQTchmCS,https://api.spotify.com/v1/tracks/0RcGrOHL67qY...,https://api.spotify.com/v1/audio-analysis/0RcG...,482125,4
67,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,3EfldjXLMDUU6zYUAfEoGz,spotify:track:3EfldjXLMDUU6zYUAfEoGz,https://api.spotify.com/v1/tracks/3EfldjXLMDUU...,https://api.spotify.com/v1/audio-analysis/3Efl...,309600,4
68,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,4ClJlECmc0Yfxz6IiI3sye,spotify:track:4ClJlECmc0Yfxz6IiI3sye,https://api.spotify.com/v1/tracks/4ClJlECmc0Yf...,https://api.spotify.com/v1/audio-analysis/4ClJ...,309600,4
69,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,5Lv60aUIxDRhnNXrcECgit,spotify:track:5Lv60aUIxDRhnNXrcECgit,https://api.spotify.com/v1/tracks/5Lv60aUIxDRh...,https://api.spotify.com/v1/audio-analysis/5Lv6...,309600,4


In order not to overload the server with the requests, we need to split the data into chunks and add pauses between the requests:

In [12]:
def split_dataframe(df, chunk_size = 50): 
    chunks = []
    num_chunks = math.ceil(len(df) / chunk_size)
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

Checking this function:

In [13]:
song_uris_splited = split_dataframe(song_uris, chunk_size = 50)
song_uris_splited

[['spotify:track:0g7cOz3g27daGHxymyOEHt',
  'spotify:track:0RcGrOHL67qY8doQTchmCS',
  'spotify:track:3EfldjXLMDUU6zYUAfEoGz',
  'spotify:track:4ClJlECmc0Yfxz6IiI3sye',
  'spotify:track:5Lv60aUIxDRhnNXrcECgit',
  'spotify:track:198Kj3dapHMkGttkRPdlPZ',
  'spotify:track:0RcGrOHL67qY8doQTchmCS',
  'spotify:track:3EfldjXLMDUU6zYUAfEoGz',
  'spotify:track:4ClJlECmc0Yfxz6IiI3sye',
  'spotify:track:5Lv60aUIxDRhnNXrcECgit',
  'spotify:track:198Kj3dapHMkGttkRPdlPZ',
  'spotify:track:0RcGrOHL67qY8doQTchmCS',
  'spotify:track:3EfldjXLMDUU6zYUAfEoGz',
  'spotify:track:4ClJlECmc0Yfxz6IiI3sye',
  'spotify:track:5Lv60aUIxDRhnNXrcECgit',
  'spotify:track:198Kj3dapHMkGttkRPdlPZ',
  'spotify:track:0RcGrOHL67qY8doQTchmCS',
  'spotify:track:3EfldjXLMDUU6zYUAfEoGz',
  'spotify:track:4ClJlECmc0Yfxz6IiI3sye',
  'spotify:track:5Lv60aUIxDRhnNXrcECgit',
  'spotify:track:198Kj3dapHMkGttkRPdlPZ',
  'spotify:track:0RcGrOHL67qY8doQTchmCS',
  'spotify:track:3EfldjXLMDUU6zYUAfEoGz',
  'spotify:track:4ClJlECmc0Yfxz6Ii

Adding a function for feature extraction, using the chunks:

In [14]:
def get_audio_features_chunks(chunks):
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))
    audio_features = pd.DataFrame()
    for chunk in chunks:
        results = sp.audio_features(chunk)
        audio_features = pd.concat([audio_features, pd.DataFrame(results)])
        print('I am doing something with the chunk')
        print('I am taking respectful nap for 20 seconds :)')
        time.sleep(20)
    return audio_features

In [15]:
# testing the function get_audio_features

get_audio_features_chunks(song_uris_splited)

I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,0g7cOz3g27daGHxymyOEHt,spotify:track:0g7cOz3g27daGHxymyOEHt,https://api.spotify.com/v1/tracks/0g7cOz3g27da...,https://api.spotify.com/v1/audio-analysis/0g7c...,309600,4
1,0.440,0.250,5,-16.942,0,0.0604,0.974,0.968000,0.128,0.166,133.807,audio_features,0RcGrOHL67qY8doQTchmCS,spotify:track:0RcGrOHL67qY8doQTchmCS,https://api.spotify.com/v1/tracks/0RcGrOHL67qY...,https://api.spotify.com/v1/audio-analysis/0RcG...,482125,4
2,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,3EfldjXLMDUU6zYUAfEoGz,spotify:track:3EfldjXLMDUU6zYUAfEoGz,https://api.spotify.com/v1/tracks/3EfldjXLMDUU...,https://api.spotify.com/v1/audio-analysis/3Efl...,309600,4
3,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,4ClJlECmc0Yfxz6IiI3sye,spotify:track:4ClJlECmc0Yfxz6IiI3sye,https://api.spotify.com/v1/tracks/4ClJlECmc0Yf...,https://api.spotify.com/v1/audio-analysis/4ClJ...,309600,4
4,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,5Lv60aUIxDRhnNXrcECgit,spotify:track:5Lv60aUIxDRhnNXrcECgit,https://api.spotify.com/v1/tracks/5Lv60aUIxDRh...,https://api.spotify.com/v1/audio-analysis/5Lv6...,309600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,0.440,0.250,5,-16.942,0,0.0604,0.974,0.968000,0.128,0.166,133.807,audio_features,0RcGrOHL67qY8doQTchmCS,spotify:track:0RcGrOHL67qY8doQTchmCS,https://api.spotify.com/v1/tracks/0RcGrOHL67qY...,https://api.spotify.com/v1/audio-analysis/0RcG...,482125,4
17,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,3EfldjXLMDUU6zYUAfEoGz,spotify:track:3EfldjXLMDUU6zYUAfEoGz,https://api.spotify.com/v1/tracks/3EfldjXLMDUU...,https://api.spotify.com/v1/audio-analysis/3Efl...,309600,4
18,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,4ClJlECmc0Yfxz6IiI3sye,spotify:track:4ClJlECmc0Yfxz6IiI3sye,https://api.spotify.com/v1/tracks/4ClJlECmc0Yf...,https://api.spotify.com/v1/audio-analysis/4ClJ...,309600,4
19,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,5Lv60aUIxDRhnNXrcECgit,spotify:track:5Lv60aUIxDRhnNXrcECgit,https://api.spotify.com/v1/tracks/5Lv60aUIxDRh...,https://api.spotify.com/v1/audio-analysis/5Lv6...,309600,4


And finally creating a function with everything in one:

In [16]:
def get_audio_features(list_of_uris):
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret= Client_Secret))
    
    chunks = split_dataframe(list_of_uris)

    audio_features = pd.DataFrame()
    for chunk in chunks:
        results = sp.audio_features(chunk)
        audio_features = pd.concat([audio_features, pd.DataFrame(results)])
        print('I am doing something with the chunk')
        print('I am taking respectful nap for 20 seconds :)')
        time.sleep(20)
    return audio_features

In [17]:
get_audio_features(song_uris)


I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,0g7cOz3g27daGHxymyOEHt,spotify:track:0g7cOz3g27daGHxymyOEHt,https://api.spotify.com/v1/tracks/0g7cOz3g27da...,https://api.spotify.com/v1/audio-analysis/0g7c...,309600,4
1,0.440,0.250,5,-16.942,0,0.0604,0.974,0.968000,0.128,0.166,133.807,audio_features,0RcGrOHL67qY8doQTchmCS,spotify:track:0RcGrOHL67qY8doQTchmCS,https://api.spotify.com/v1/tracks/0RcGrOHL67qY...,https://api.spotify.com/v1/audio-analysis/0RcG...,482125,4
2,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,3EfldjXLMDUU6zYUAfEoGz,spotify:track:3EfldjXLMDUU6zYUAfEoGz,https://api.spotify.com/v1/tracks/3EfldjXLMDUU...,https://api.spotify.com/v1/audio-analysis/3Efl...,309600,4
3,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,4ClJlECmc0Yfxz6IiI3sye,spotify:track:4ClJlECmc0Yfxz6IiI3sye,https://api.spotify.com/v1/tracks/4ClJlECmc0Yf...,https://api.spotify.com/v1/audio-analysis/4ClJ...,309600,4
4,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,5Lv60aUIxDRhnNXrcECgit,spotify:track:5Lv60aUIxDRhnNXrcECgit,https://api.spotify.com/v1/tracks/5Lv60aUIxDRh...,https://api.spotify.com/v1/audio-analysis/5Lv6...,309600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,0.440,0.250,5,-16.942,0,0.0604,0.974,0.968000,0.128,0.166,133.807,audio_features,0RcGrOHL67qY8doQTchmCS,spotify:track:0RcGrOHL67qY8doQTchmCS,https://api.spotify.com/v1/tracks/0RcGrOHL67qY...,https://api.spotify.com/v1/audio-analysis/0RcG...,482125,4
17,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,3EfldjXLMDUU6zYUAfEoGz,spotify:track:3EfldjXLMDUU6zYUAfEoGz,https://api.spotify.com/v1/tracks/3EfldjXLMDUU...,https://api.spotify.com/v1/audio-analysis/3Efl...,309600,4
18,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,4ClJlECmc0Yfxz6IiI3sye,spotify:track:4ClJlECmc0Yfxz6IiI3sye,https://api.spotify.com/v1/tracks/4ClJlECmc0Yf...,https://api.spotify.com/v1/audio-analysis/4ClJ...,309600,4
19,0.557,0.442,5,-7.224,1,0.0243,0.731,0.000015,0.110,0.213,146.277,audio_features,5Lv60aUIxDRhnNXrcECgit,spotify:track:5Lv60aUIxDRhnNXrcECgit,https://api.spotify.com/v1/tracks/5Lv60aUIxDRh...,https://api.spotify.com/v1/audio-analysis/5Lv6...,309600,4


<h2>Part 3</h2>

3) Once the previous function has been created, create another function add_audio_features(df, audio_features_df) to concat a given dataframe with the dataframe containing the audio features alongside any other desired info, and return the extended data frame. Replace the old internal files of songs (hot and not hot) with the extended data frames with the audio features and save them into separate files on the disk.

In [18]:
hot_songs = pd.read_csv('hot_songs.csv')
hot_songs.head()

,track_name,artist_name
0,anti-hero,taylor swift
1,lift me up,rihanna
2,unholy,sam smith & kim petras
3,bad habit,steve lacy
4,as it was,harry styles


In [19]:
not_hot_songs = pd.read_csv('not_hot_songs.csv')
not_hot_songs.head()

,artist_name,track_name
0,glenn miller,the little man who wasn't there
1,misfits,american psycho
2,elliott smith,somebody that i used to know
3,june carter cash,juke box blues
4,"emerson, lake & palmer","karn evil 9 1st impression, pt. 1"


Creating a list of uris for hot_songs, using function 'search_song'.

In [20]:
hot_uris = []
for i in tqdm.tqdm(range(len(hot_songs))): # to see the progress
    hot_uris.append(search_song(hot_songs['track_name'].iloc[i], hot_songs['artist_name'].iloc[i], ask_for_options=False))
    if i % 50 == 49:
        time.sleep(20)

  0%|                                                   | 0/100 [00:00<?, ?it/s]

Searching for song anti-hero of artist taylor swift


  2%|▊                                          | 2/100 [00:00<00:22,  4.28it/s]

Searching for song lift me up of artist rihanna
Searching for song unholy of artist sam smith & kim petras


  4%|█▋                                         | 4/100 [00:00<00:16,  5.93it/s]

...

 98%|█████████████████████████████████████████▏| 98/100 [00:37<00:00,  5.90it/s]

Searching for song country on of artist luke bryan
Searching for song bzrp music sessions, vol. 52 of artist bizarrap & quevedo


 99%|█████████████████████████████████████████▌| 99/100 [00:37<00:00,  6.59it/s]

Searching for song evergreen of artist omar apollo


100%|█████████████████████████████████████████| 100/100 [00:57<00:00,  1.73it/s]


In [21]:
hot_uris

['spotify:track:0V3wPSX9ygBnCm8psDIegu',
 'spotify:track:35ovElsgyAtQwYPYnZJECg',
 '',
 'spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK',
 'spotify:track:4LRPiXqCikLlN15c3yImP7',
 'spotify:track:5jQI2r1RdgtuT8S3iG8zFC',
 'spotify:track:3rWDp9tBPQR9z6U5YyRSK4',
 '',
 'spotify:track:0jvo9CjnbR0lYUDTSNTMiu',
 'spotify:track:4C6Uex2ILwJi9sZXRdmqXp',
 'spotify:track:34ZAzO78a5DAVNrYIGWcPm',
 'spotify:track:3eX0NZfLtGzoLUxPNvRfqm',
 'spotify:track:4h9wh7iOZ0GGn8QVp4RAOB',
 'spotify:track:5uU1uuyaUIBOiiUSQG7Wl3',
 '',
 '',
 'spotify:track:7KokYm8cMIXCsGVmUvKtqf',
 'spotify:track:0hquQWY3xvYqN4qtiquniF',
 '',
 '',
 'spotify:track:5IgjP7X4th6nMNDh4akUHb',
 'spotify:track:3cBsEDNhFI9E82vPj3kvi3',
 'spotify:track:2RJAKIw6nIkgZVsAIKhmqz',
 'spotify:track:1xwAWUI6Dj0WGC3KiUPN0O',
 '',
 'spotify:track:20efeySIfZoiSaISGLBbNs',
 'spotify:track:2Ch7LmS7r2Gy2kc64wv3Bz',
 '',
 'spotify:track:3WMj8moIAXJhHsyLaqIIHI',
 'spotify:track:1IHWl5LamUGEuP4ozKQSXZ',
 'spotify:track:1xzi1Jcr7mEi9K2RfzLOqS',
 'spotify:track:6

Adding the result as an additional column to a copy of the DataFrame.

In [22]:
hot_songs_new = hot_songs.copy()
hot_songs_new['uri_'] = hot_uris
hot_songs_new.head()

,track_name,artist_name,uri_
0,anti-hero,taylor swift,spotify:track:0V3wPSX9ygBnCm8psDIegu
1,lift me up,rihanna,spotify:track:35ovElsgyAtQwYPYnZJECg
2,unholy,sam smith & kim petras,
3,bad habit,steve lacy,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK
4,as it was,harry styles,spotify:track:4LRPiXqCikLlN15c3yImP7


Getting rid of empty strings (it means that this song wasn't found on Spotify).

In [23]:
hot_songs_new = hot_songs_new[hot_songs_new['uri_'] != '']
hot_songs_new.head()

,track_name,artist_name,uri_
0,anti-hero,taylor swift,spotify:track:0V3wPSX9ygBnCm8psDIegu
1,lift me up,rihanna,spotify:track:35ovElsgyAtQwYPYnZJECg
3,bad habit,steve lacy,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK
4,as it was,harry styles,spotify:track:4LRPiXqCikLlN15c3yImP7
5,lavender haze,taylor swift,spotify:track:5jQI2r1RdgtuT8S3iG8zFC


Using function 'get_audio_features' to get audio features of all hot songs.

In [24]:
audio_features = get_audio_features(hot_songs_new['uri_'])

I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)


In [25]:
hot_songs_new.reset_index(drop=True)

,track_name,artist_name,uri_
0,anti-hero,taylor swift,spotify:track:0V3wPSX9ygBnCm8psDIegu
1,lift me up,rihanna,spotify:track:35ovElsgyAtQwYPYnZJECg
2,bad habit,steve lacy,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK
3,as it was,harry styles,spotify:track:4LRPiXqCikLlN15c3yImP7
4,lavender haze,taylor swift,spotify:track:5jQI2r1RdgtuT8S3iG8zFC
...,...,...,...
71,betty (get money),yung gravy,spotify:track:4cacyP5c3PMlfnyjpg13xW
72,300 blackout,kodak black,spotify:track:3y3NwQPQAjKCunRz2eKggA
73,dear reader,taylor swift,spotify:track:3QF5RsWzK1lCvf2o2cY65P
74,country on,luke bryan,spotify:track:1tRxwf8Q0AcshfHuaD86Yt


In [26]:
audio_features.reset_index(drop=True)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.637,0.643,4,-6.571,1,0.0519,0.13000,0.000002,0.1420,0.5330,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
1,0.247,0.299,9,-6.083,1,0.0315,0.89900,0.000000,0.1310,0.1720,177.115,audio_features,35ovElsgyAtQwYPYnZJECg,spotify:track:35ovElsgyAtQwYPYnZJECg,https://api.spotify.com/v1/tracks/35ovElsgyAtQ...,https://api.spotify.com/v1/audio-analysis/35ov...,196520,4
2,0.686,0.507,1,-7.097,1,0.0357,0.62600,0.000075,0.3830,0.6900,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
3,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.6620,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
4,0.733,0.436,10,-10.489,1,0.0800,0.25800,0.000573,0.1570,0.0976,96.985,audio_features,5jQI2r1RdgtuT8S3iG8zFC,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,https://api.spotify.com/v1/tracks/5jQI2r1Rdgtu...,https://api.spotify.com/v1/audio-analysis/5jQI...,202396,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.723,0.772,1,-6.784,1,0.0792,0.00524,0.000012,0.3670,0.5800,102.004,audio_features,4cacyP5c3PMlfnyjpg13xW,spotify:track:4cacyP5c3PMlfnyjpg13xW,https://api.spotify.com/v1/tracks/4cacyP5c3PMl...,https://api.spotify.com/v1/audio-analysis/4cac...,146471,4
72,0.669,0.741,0,-6.545,0,0.3160,0.01980,0.000000,0.2230,0.3960,167.903,audio_features,3y3NwQPQAjKCunRz2eKggA,spotify:track:3y3NwQPQAjKCunRz2eKggA,https://api.spotify.com/v1/tracks/3y3NwQPQAjKC...,https://api.spotify.com/v1/audio-analysis/3y3N...,166494,4
73,0.562,0.388,0,-12.088,1,0.0705,0.48100,0.001450,0.1170,0.1590,107.747,audio_features,3QF5RsWzK1lCvf2o2cY65P,spotify:track:3QF5RsWzK1lCvf2o2cY65P,https://api.spotify.com/v1/tracks/3QF5RsWzK1lC...,https://api.spotify.com/v1/audio-analysis/3QF5...,225194,4
74,0.520,0.751,5,-5.064,1,0.0551,0.17100,0.000007,0.0624,0.5190,156.044,audio_features,1tRxwf8Q0AcshfHuaD86Yt,spotify:track:1tRxwf8Q0AcshfHuaD86Yt,https://api.spotify.com/v1/tracks/1tRxwf8Q0Acs...,https://api.spotify.com/v1/audio-analysis/1tRx...,236456,4


Concatenating all hot songs (song names and artist's name) with the extracted audio features.

In [27]:
hot_songs_with_audio_features = pd.concat([hot_songs_new.reset_index(drop=True), audio_features.reset_index(drop=True)], axis=1)
hot_songs_with_audio_features.head()

,track_name,artist_name,uri_,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,anti-hero,taylor swift,spotify:track:0V3wPSX9ygBnCm8psDIegu,0.637,0.643,4,-6.571,1,0.0519,0.130,...,0.142,0.5330,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
1,lift me up,rihanna,spotify:track:35ovElsgyAtQwYPYnZJECg,0.247,0.299,9,-6.083,1,0.0315,0.899,...,0.131,0.1720,177.115,audio_features,35ovElsgyAtQwYPYnZJECg,spotify:track:35ovElsgyAtQwYPYnZJECg,https://api.spotify.com/v1/tracks/35ovElsgyAtQ...,https://api.spotify.com/v1/audio-analysis/35ov...,196520,4
2,bad habit,steve lacy,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,0.686,0.507,1,-7.097,1,0.0357,0.626,...,0.383,0.6900,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
3,as it was,harry styles,spotify:track:4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,0.0557,0.342,...,0.311,0.6620,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
4,lavender haze,taylor swift,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,0.733,0.436,10,-10.489,1,0.0800,0.258,...,0.157,0.0976,96.985,audio_features,5jQI2r1RdgtuT8S3iG8zFC,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,https://api.spotify.com/v1/tracks/5jQI2r1Rdgtu...,https://api.spotify.com/v1/audio-analysis/5jQI...,202396,4


Getting rid of the duplicated 'uri' column.

In [28]:
hot_songs_with_audio_features = hot_songs_with_audio_features.drop(['uri_'], axis=1)

In [29]:
hot_songs_with_audio_features

,track_name,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,anti-hero,taylor swift,0.637,0.643,4,-6.571,1,0.0519,0.13000,0.000002,0.1420,0.5330,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
1,lift me up,rihanna,0.247,0.299,9,-6.083,1,0.0315,0.89900,0.000000,0.1310,0.1720,177.115,audio_features,35ovElsgyAtQwYPYnZJECg,spotify:track:35ovElsgyAtQwYPYnZJECg,https://api.spotify.com/v1/tracks/35ovElsgyAtQ...,https://api.spotify.com/v1/audio-analysis/35ov...,196520,4
2,bad habit,steve lacy,0.686,0.507,1,-7.097,1,0.0357,0.62600,0.000075,0.3830,0.6900,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
3,as it was,harry styles,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.6620,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
4,lavender haze,taylor swift,0.733,0.436,10,-10.489,1,0.0800,0.25800,0.000573,0.1570,0.0976,96.985,audio_features,5jQI2r1RdgtuT8S3iG8zFC,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,https://api.spotify.com/v1/tracks/5jQI2r1Rdgtu...,https://api.spotify.com/v1/audio-analysis/5jQI...,202396,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,betty (get money),yung gravy,0.723,0.772,1,-6.784,1,0.0792,0.00524,0.000012,0.3670,0.5800,102.004,audio_features,4cacyP5c3PMlfnyjpg13xW,spotify:track:4cacyP5c3PMlfnyjpg13xW,https://api.spotify.com/v1/tracks/4cacyP5c3PMl...,https://api.spotify.com/v1/audio-analysis/4cac...,146471,4
72,300 blackout,kodak black,0.669,0.741,0,-6.545,0,0.3160,0.01980,0.000000,0.2230,0.3960,167.903,audio_features,3y3NwQPQAjKCunRz2eKggA,spotify:track:3y3NwQPQAjKCunRz2eKggA,https://api.spotify.com/v1/tracks/3y3NwQPQAjKC...,https://api.spotify.com/v1/audio-analysis/3y3N...,166494,4
73,dear reader,taylor swift,0.562,0.388,0,-12.088,1,0.0705,0.48100,0.001450,0.1170,0.1590,107.747,audio_features,3QF5RsWzK1lCvf2o2cY65P,spotify:track:3QF5RsWzK1lCvf2o2cY65P,https://api.spotify.com/v1/tracks/3QF5RsWzK1lC...,https://api.spotify.com/v1/audio-analysis/3QF5...,225194,4
74,country on,luke bryan,0.520,0.751,5,-5.064,1,0.0551,0.17100,0.000007,0.0624,0.5190,156.044,audio_features,1tRxwf8Q0AcshfHuaD86Yt,spotify:track:1tRxwf8Q0AcshfHuaD86Yt,https://api.spotify.com/v1/tracks/1tRxwf8Q0Acs...,https://api.spotify.com/v1/audio-analysis/1tRx...,236456,4


Now, we can create a function:

In [30]:
def add_audio_features(df, audio_features):
    df_with_features = pd.concat([df.reset_index(drop=True), audio_features.reset_index(drop=True)], axis=1)
    return df_with_features.drop(['uri_'], axis=1)
    

In [31]:
add_audio_features(hot_songs_new, audio_features)

,track_name,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,anti-hero,taylor swift,0.637,0.643,4,-6.571,1,0.0519,0.13000,0.000002,0.1420,0.5330,97.008,audio_features,0V3wPSX9ygBnCm8psDIegu,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690,4
1,lift me up,rihanna,0.247,0.299,9,-6.083,1,0.0315,0.89900,0.000000,0.1310,0.1720,177.115,audio_features,35ovElsgyAtQwYPYnZJECg,spotify:track:35ovElsgyAtQwYPYnZJECg,https://api.spotify.com/v1/tracks/35ovElsgyAtQ...,https://api.spotify.com/v1/audio-analysis/35ov...,196520,4
2,bad habit,steve lacy,0.686,0.507,1,-7.097,1,0.0357,0.62600,0.000075,0.3830,0.6900,168.955,audio_features,5CM4UuQ9Gnd6K2YyKGPMoK,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,https://api.spotify.com/v1/tracks/5CM4UuQ9Gnd6...,https://api.spotify.com/v1/audio-analysis/5CM4...,232115,4
3,as it was,harry styles,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.6620,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
4,lavender haze,taylor swift,0.733,0.436,10,-10.489,1,0.0800,0.25800,0.000573,0.1570,0.0976,96.985,audio_features,5jQI2r1RdgtuT8S3iG8zFC,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,https://api.spotify.com/v1/tracks/5jQI2r1Rdgtu...,https://api.spotify.com/v1/audio-analysis/5jQI...,202396,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,betty (get money),yung gravy,0.723,0.772,1,-6.784,1,0.0792,0.00524,0.000012,0.3670,0.5800,102.004,audio_features,4cacyP5c3PMlfnyjpg13xW,spotify:track:4cacyP5c3PMlfnyjpg13xW,https://api.spotify.com/v1/tracks/4cacyP5c3PMl...,https://api.spotify.com/v1/audio-analysis/4cac...,146471,4
72,300 blackout,kodak black,0.669,0.741,0,-6.545,0,0.3160,0.01980,0.000000,0.2230,0.3960,167.903,audio_features,3y3NwQPQAjKCunRz2eKggA,spotify:track:3y3NwQPQAjKCunRz2eKggA,https://api.spotify.com/v1/tracks/3y3NwQPQAjKC...,https://api.spotify.com/v1/audio-analysis/3y3N...,166494,4
73,dear reader,taylor swift,0.562,0.388,0,-12.088,1,0.0705,0.48100,0.001450,0.1170,0.1590,107.747,audio_features,3QF5RsWzK1lCvf2o2cY65P,spotify:track:3QF5RsWzK1lCvf2o2cY65P,https://api.spotify.com/v1/tracks/3QF5RsWzK1lC...,https://api.spotify.com/v1/audio-analysis/3QF5...,225194,4
74,country on,luke bryan,0.520,0.751,5,-5.064,1,0.0551,0.17100,0.000007,0.0624,0.5190,156.044,audio_features,1tRxwf8Q0AcshfHuaD86Yt,spotify:track:1tRxwf8Q0AcshfHuaD86Yt,https://api.spotify.com/v1/tracks/1tRxwf8Q0Acs...,https://api.spotify.com/v1/audio-analysis/1tRx...,236456,4


Putting everything together:

In [32]:
def add_audio_features(df):
        
    song_uris = []
    for i in tqdm.tqdm(range(len(df))): # to see the progress
        song_uris.append(search_song(df['track_name'].iloc[i], df['artist_name'].iloc[i], ask_for_options=False))
        if i % 50 == 49:
            time.sleep(20)

    df_new = df.copy()
    df_new['uri_'] = song_uris

    df_new = df_new[df_new['uri_'] != '']

    audio_features = get_audio_features(df_new['uri_'])

    df_new_with_audio_features = pd.concat([df_new.reset_index(drop=True), audio_features.reset_index(drop=True)], axis=1)

    df_new_with_audio_features = df_new_with_audio_features.drop(['uri_'], axis=1)

    return df_new_with_audio_features

In [33]:
not_hot_songs.head()

,artist_name,track_name
0,glenn miller,the little man who wasn't there
1,misfits,american psycho
2,elliott smith,somebody that i used to know
3,june carter cash,juke box blues
4,"emerson, lake & palmer","karn evil 9 1st impression, pt. 1"


In [34]:
not_hot_songs.isna().sum()

artist_name    0
track_name     0
dtype: int64

In [35]:
not_hot_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  4000 non-null   object
 1   track_name   4000 non-null   object
dtypes: object(2)
memory usage: 62.6+ KB


In [36]:
not_hot_songs[not_hot_songs['artist_name'] == '']

,artist_name,track_name


In [37]:
not_hot_songs[not_hot_songs['track_name'] == '']

,artist_name,track_name


In [38]:
hot_songs_with_audio_features = add_audio_features(hot_songs)
hot_songs_with_audio_features.to_csv('hot_songs_with_audio_features.csv', index=False)


  0%|                                                   | 0/100 [00:00<?, ?it/s]

Searching for song anti-hero of artist taylor swift


  2%|▊                                          | 2/100 [00:00<00:17,  5.70it/s]

Searching for song lift me up of artist rihanna
Searching for song unholy of artist sam smith & kim petras


  4%|█▋                                         | 4/100 [00:00<00:13,  7.19it/s]

...

Searching for song country on of artist luke bryan
Searching for song bzrp music sessions, vol. 52 of artist bizarrap & quevedo


 99%|█████████████████████████████████████████▌| 99/100 [00:34<00:00,  8.22it/s]

Searching for song evergreen of artist omar apollo


100%|█████████████████████████████████████████| 100/100 [00:54<00:00,  1.84it/s]


I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)


In [39]:
not_hot_songs_with_audio_features = add_audio_features(not_hot_songs)
not_hot_songs_with_audio_features.to_csv('not_hot_songs_with_audio_features.csv', index=False)

  0%|                                                  | 0/4000 [00:00<?, ?it/s]

Searching for song the little man who wasn't there of artist glenn miller


  0%|                                          | 1/4000 [00:00<14:26,  4.62it/s]

Searching for song american psycho of artist misfits


  0%|                                          | 3/4000 [00:00<14:30,  4.59it/s]

...

Searching for song stand up (remastered) of artist the band
Searching for song this way of artist khalid


100%|██████████████████████████████████████▉| 3999/4000 [38:50<00:00,  4.45it/s]

Searching for song big city life of artist mattafix
Searching for song release me of artist conway twitty


100%|███████████████████████████████████████| 4000/4000 [39:10<00:00,  1.70it/s]


I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with the chunk
I am taking respectful nap for 20 seconds :)
I am doing something with th

In [40]:
hot_songs_with_audio_features['type'] = 'hot'
not_hot_songs_with_audio_features['type'] = 'not_hot'

In [41]:
final_dataset = pd.concat([hot_songs_with_audio_features, not_hot_songs_with_audio_features], axis = 0)
final_dataset.to_csv('final_dataset.csv', index=False)

In [42]:
hot_songs_with_audio_features.shape

(76, 20)

In [43]:
not_hot_songs_with_audio_features.shape

(3786, 20)

In [44]:
final_dataset.shape

(3862, 20)